<a href="https://colab.research.google.com/github/yushendye/AerialInspection/blob/main/detectron_training_10kepochs_18_aug.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# install dependencies: (use cu101 because colab has CUDA 10.1)
!pip install -U torch==1.5 torchvision==0.6 -f https://download.pytorch.org/whl/cu101/torch_stable.html 
!pip install cython pyyaml==5.1
!pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
!gcc --version
# opencv is pre-installed on colab

Looking in links: https://download.pytorch.org/whl/cu101/torch_stable.html
     |████████████████████████████████| 703.8 MB 23 kB/s 
     |████████████████████████████████| 6.6 MB 21.8 MB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.9.0+cu102
    Uninstalling torch-1.9.0+cu102:
      Successfully uninstalled torch-1.9.0+cu102
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.10.0+cu102
    Uninstalling torchvision-0.10.0+cu102:
      Successfully uninstalled torchvision-0.10.0+cu102
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtext 0.10.0 requires torch==1.9.0, but you have torch 1.5.0+cu101 which is incompatible.
  Cloning https://github.com/cocodataset/cocoapi.git to /tmp/pip-req-build-b74zjy_9
  Running command git clone -q https://github.com/cocodataset/cocoapi.git /tmp/pip-req-build-b74z

In [ ]:
!pip install detectron2==0.1.3 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.5/index.html

Looking in links: https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.5/index.html
     |████████████████████████████████| 6.2 MB 862 kB/s 
  Attempting uninstall: detectron2
    Found existing installation: detectron2 0.5+cu102
    Uninstalling detectron2-0.5+cu102:
      Successfully uninstalled detectron2-0.5+cu102


In [ ]:
# check pytorch installation: 
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
assert torch.__version__.startswith("1.9")   # please manually install torch 1.9 if Colab changes its default version

In [ ]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

In [10]:
%cd /content/
!gdown --id 1Hh7XFuyPV2qH5Pg6sqce6WmdOZoSh-th
!gdown --id 1FeJJYBgcfuZsunkk-WQFkHO8ry-Ywhrn

!unzip /content/Dataset.zip

/content
Downloading...
From: https://drive.google.com/uc?id=1Hh7XFuyPV2qH5Pg6sqce6WmdOZoSh-th
To: /content/main.json
100% 1.54M/1.54M [00:00<00:00, 50.0MB/s]
Downloading...
From: https://drive.google.com/uc?id=1FeJJYBgcfuZsunkk-WQFkHO8ry-Ywhrn
To: /content/Dataset.zip
114MB [00:03, 37.0MB/s]
Archive:  /content/Dataset.zip
   creating: Dataset/
  inflating: Dataset/000000010363.jpg  
  inflating: Dataset/000000014380.jpg  
  inflating: Dataset/000000018837.jpg  
  inflating: Dataset/000000022755.jpg  
  inflating: Dataset/000000026926.jpg  
  inflating: Dataset/000000030828.jpg  
  inflating: Dataset/000000031118.jpg  
  inflating: Dataset/000000032941.jpg  
  inflating: Dataset/000000033109.jpg  
  inflating: Dataset/000000033221.jpg  
  inflating: Dataset/000000045596.jpg  
  inflating: Dataset/000000057597.jpg  
  inflating: Dataset/000000057672.jpg  
  inflating: Dataset/000000067616.jpg  
  inflating: Dataset/000000068093.jpg  
  inflating: Dataset/000000074058.jpg  
  inflating: 

In [11]:
# if your dataset is in COCO format, this cell can be replaced by the following three lines:
from detectron2.data.datasets import register_coco_instances
register_coco_instances("my_dataset_train2", {}, "/content/main.json", "/content/Dataset")
#register_coco_instances("my_dataset_val", {}, "/content/drive/MyDrive/detectron_train/test/test.json", "/content/drive/MyDrive/detectron_train/test")

In [17]:
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
import os

#!mkdir /content/drive/MyDrive/18_Aug_Dataset
#!cp -r /content/Dataset /content/drive/MyDrive/18_Aug_Dataset
#!cp /content/main.json /content/drive/MyDrive/18_Aug_Dataset

cfg = get_cfg()
cfg.OUTPUT_DIR = "/content/drive/MyDrive/18_Aug_Detectron"
cfg.SOLVER.CHECKPOINT_PERIOD = 100
cfg.merge_from_file(model_zoo.get_config_file("Misc/cascade_mask_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("my_dataset_train2",)
#cfg.DATASETS.TEST = ("my_dataset_train",)
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("Misc/cascade_mask_rcnn_R_50_FPN_3x.yaml")  # Let training initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00025  # pick a good LR
cfg.SOLVER.MAX_ITER = 10000    # 300 iterations seems good enough for this toy dataset; you will need to train longer for a practical dataset
cfg.SOLVER.STEPS = []        # do not decay learning rate
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128   # faster, and good enough for this toy dataset (default: 512)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 6  # only has one class (ballon). (see https://detectron2.readthedocs.io/tutorials/datasets.html#update-the-config-for-new-datasets)
# NOTE: this config means the number of classes, but a few popular unofficial tutorials incorrect uses num_classes+1 here.

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=False)
trainer.train()
trainer.test()

[08/18 08:56:55 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

Skip loading parameter 'roi_heads.box_predictor.0.cls_score.weight' to the model due to incompatible shapes: (81, 1024) in the checkpoint but (7, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.0.cls_score.bias' to the model due to incompatible shapes: (81,) in the checkpoint but (7,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.1.cls_score.weight' to the model due to incompatible shapes: (81, 1024) in the checkpoint but (7, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.1.cls_score.bias' to the model due to incompatible shapes: (81,) in the checkpoint but (7,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.2.cls_score.weight' to the model due to incompatible shapes: (81, 1024) in the checkpoint but (7, 1024) 

[08/18 08:56:56 d2.engine.train_loop]: Starting training from iteration 0
[08/18 08:57:24 d2.utils.events]:  eta: 3:57:23  iter: 19  total_loss: 7.807  loss_cls_stage0: 2.002  loss_box_reg_stage0: 0.321  loss_cls_stage1: 1.832  loss_box_reg_stage1: 0.520  loss_cls_stage2: 1.910  loss_box_reg_stage2: 0.512  loss_mask: 0.691  loss_rpn_cls: 0.042  loss_rpn_loc: 0.025  time: 1.4321  data_time: 0.0287  lr: 0.000005  max_mem: 7441M
[08/18 08:57:53 d2.utils.events]:  eta: 3:59:50  iter: 39  total_loss: 7.571  loss_cls_stage0: 1.939  loss_box_reg_stage0: 0.301  loss_cls_stage1: 1.778  loss_box_reg_stage1: 0.467  loss_cls_stage2: 1.865  loss_box_reg_stage2: 0.565  loss_mask: 0.691  loss_rpn_cls: 0.024  loss_rpn_loc: 0.012  time: 1.4333  data_time: 0.0112  lr: 0.000010  max_mem: 7441M
[08/18 08:58:22 d2.utils.events]:  eta: 3:59:05  iter: 59  total_loss: 7.533  loss_cls_stage0: 1.853  loss_box_reg_stage0: 0.277  loss_cls_stage1: 1.674  loss_box_reg_stage1: 0.510  loss_cls_stage2: 1.793  loss_box

FileNotFoundError: ignored